In [28]:
import igl
import numpy as np
import meshplot as mp

from collections import deque

In [29]:
# build graph of geodesic distances on a mesh
def build_geodesic_graph(V, F):
    n = V.shape[0]  # number of vertices
    adj = [[] for _ in range(n)]  # adjacency list: for each vertex, list of (neighbor, weight)

    # Compute triangle-triangle adjacency for dual edges
    TT, _ = igl.triangle_triangle_adjacency(F)

    # For each face
    for f_idx in range(F.shape[0]):
        tri = F[f_idx] 

        # For each corner of the face
        for loc in range(3):
            v  = tri[loc]            # current vertex index
            w1 = tri[(loc + 1) % 3]  # next vertex index around the triangle
            w2 = tri[(loc + 2) % 3]  # other vertex index

            # Primal edges
            for w in (w1, w2):
                d = np.linalg.norm(V[v] - V[w])
                # add primal edge
                adj[v].append((w, d, None)) # (neighbor, weight, dual edge)
                adj[w].append((v, d, None)) # undirected graph, so add both directions

            # Dual edges
            f2 = int(TT[f_idx, loc])  # adjacent face across edge opposite v
            if f2 < 0:
                # boundary edge: no adjacent face
                continue

            # find the vertex in face f2 that is not w1 or w2
            edge_vs = {w1, w2}
            vp = next(int(x) for x in F[f2] if int(x) not in edge_vs)

            # Compute vectors for angle calculations at shared edge vertex w1
            v_w1_v  = V[v]  - V[w1]    # vector w1->v
            v_w1_w2 = V[w2] - V[w1]    # vector w1->w2 (edge direction)
            v_w1_vp = V[vp] - V[w1]    # vector w1->vp (across triangle f2)

            # norms of these vectors
            n_w1_v  = np.linalg.norm(v_w1_v)
            n_w1_w2 = np.linalg.norm(v_w1_w2)
            n_w1_vp = np.linalg.norm(v_w1_vp)

            # corner angles at w1 in both triangles
            alfa = np.arccos(np.clip(np.dot(v_w1_v, v_w1_w2) / (n_w1_v * n_w1_w2), -1, 1))
            beta = np.arccos(np.clip(np.dot(v_w1_vp, v_w1_w2) / (n_w1_vp * n_w1_w2), -1, 1))

            # Geodesic length between v and vp via w1
            dual_len = np.sqrt(n_w1_v**2 + n_w1_vp**2 - 2 * n_w1_v * n_w1_vp * np.cos(alfa + beta))

            # compute intersection point of the dual edge with the primal edge w1-w2
            # TBD
            intersection = None  # Placeholder for intersection point calculation
            # add dual edge
            adj[v].append((vp, dual_len, intersection))  # (neighbor, weight, intersection point)
            adj[vp].append((v, dual_len, intersection))  # undirected graph, so add both directions
    return adj

# plot intersection points on the mesh
def plot_intersections(V, F, adj):
    # Extract intersection points from adjacency list
    intersection_points = []
    for neighbors in adj:
        for _, _, intersection in neighbors:
            if intersection is not None:
                intersection_points.append(intersection)
    # If no intersection points found, return empty plot
    if not intersection_points:
        return mp.plot(V, F, shading={"wireframe": True})

    # Convert to numpy array
    intersection_points = np.array(intersection_points)

    # Create mesh plot
    p = mp.plot(V, F, shading={"wireframe": True})
    p.add_points(intersection_points, shading={"point_size": 0.1, "color": "red"})
    return p

# Compute geodesic distances from sources
def geodesic_distances(adj, sources):
    n = len(adj)
    prev = [None] * n  # to store the previous vertex in the path
    dist = np.full(n, np.inf)  # initialize all distances to infinity
    Q = deque()                # deque for SLF-LLL

    # Initialize queue with all source vertices at distance 0
    for s in sources:
        dist[s] = 0.0
        Q.append(s)

    # Propagate distances
    while Q:
        u = Q.popleft()
        du = dist[u]
        # relax all outgoing edges from u
        for v, w, intersection in adj[u]:
            alt = du + w
            if alt < dist[v]:
                dist[v] = alt
                prev[v] = (u, intersection)
                # SLF–LLL: push to front if smaller than current front
                if Q and alt < dist[Q[0]]:
                    Q.appendleft(v)
                else:
                    Q.append(v)

    return dist, prev


In [30]:
# extract path from target to a source
def extract_geodesic_path(target, sources, prev, V):
    path = []
    current = target
    while current is not None:
        path.append(V[current])
        if current in sources:
            break  # stop if we reach a source
        current, intersection = prev[current]  # get previous vertex and intersection point
        if intersection is not None:
            # dual edge intersection point
            path.append(intersection)    
    return path

def extract_geodesic_path_grad(target, sources, dist, V, F):
    path = []


    
    return path
    

def plot_geodesic_path(V, F, dist, path, src, target):
    p = mp.plot(V, F, c=dist, shading={"wireframe": False})

    # mark sources
    p.add_points(V[src, :], c=np.array([[1.0, 0.0, 0.0]]), shading={"point_size": 0.02})
    # mark target
    p.add_points(V[[target], :], c=np.array([[0.0, 1.0, 0.0]]), shading={"point_size": 0.02})

    # draw path
    if len(path) >= 2:
        for i in range(len(path) - 1):
            a = path[i]
            b = path[i+1]
            p.add_lines(a, b, shading={"line_width": 0.05})

    return p


In [31]:
# Load the test mesh
V, F = igl.read_triangle_mesh("./data/bunny_1k.obj")
#mp.plot(V, F, shading={"wireframe": True})

In [32]:
# compute geodesic distances
adj = build_geodesic_graph(V, F)

src = [0]

dist, prev = geodesic_distances(adj, src)

# apply color map
plotter = mp.plot(V, F, c=dist, shading={"wireframe": False})

# mark the source with a red point
plotter.add_points(V[[src], :], c=np.array([[1.0, 0.0, 0.0]]), shading={"point_size": 0.1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001504…

1

In [33]:
plot_intersections(V, F, adj)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001504…

In [35]:
src = [0]
tgt = 7

# Extraxt geodesic path from target to source
path = extract_geodesic_path(tgt, src, prev, V)

# Visualize the geodesic path
plot_geodesic_path(V, F, dist, path, src, tgt)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001504…